[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/juansensio/blog/blob/master/072_pytorch_ngc/072_pytorch_ngc.ipynb)

# Pytorch NGC

En los posts anteriores hemos visto muchos trucos para optimizar nuestro código en Pytorch, sin embargo no nos hemos preocupado por la instalación del mismo. En este post vamos a aprender a usar una versión de `Pytorch` optimizada que nos dará un extra de performance.

- https://ngc.nvidia.com/catalog/containers/nvidia:pytorch

- https://docs.docker.com/engine/install/ubuntu/

- https://github.com/NVIDIA/nvidia-docker

  docker run --gpus all --ipc=host --rm -v local_dir:container_dir nvcr.io/nvidia/pytorch:xx.xx-py3


docker run --gpus all --ipc=host --rm  nvcr.io/nvidia/pytorch:21.06-py3 echo "hola"

docker run --gpus all --ipc=host --rm  nvcr.io/nvidia/pytorch:21.06-py3 pwd

docker run --gpus all --ipc=host --rm -v $PWD/072_pytorch_ngc:/workspace  -p 8888:8888 nvcr.io/nvidia/pytorch:21.06-py3 jupyter notebook --allow-root --ip=0.0.0.0 --no-browser --NotebookApp.token=abc123

```
docker run --gpus all --ipc=host --rm -v $PWD/072_pytorch_ngc:/workspace -v $PWD/072_pytorch_ngc/data:/workspace/data  -p 8888:8888 nvcr.io/nvidia/pytorch:21.06-py3 jupyter notebook --allow-root --ip=0.0.0.0 --no-browser --NotebookApp.token=abc123
```

In [1]:
import torch 

torch.__version__

'1.9.0a0+c3d40fd'

In [2]:
torch.cuda.is_available()

True

In [3]:
import os
from sklearn.model_selection import train_test_split

def setup(path='./data', test_size=0.2, random_state=42):

    classes = sorted(os.listdir(path))

    print("Generating images and labels ...")
    images, encoded = [], []
    for ix, label in enumerate(classes):
        _images = os.listdir(f'{path}/{label}')
        images += [f'{path}/{label}/{img}' for img in _images]
        encoded += [ix]*len(_images)
    print(f'Number of images: {len(images)}')

     # train / val split
    print("Generating train / val splits ...")
    train_images, val_images, train_labels, val_labels = train_test_split(
        images,
        encoded,
        stratify=encoded,
        test_size=test_size,
        random_state=random_state
    )

    print("Training samples: ", len(train_labels))
    print("Validation samples: ", len(val_labels))
    
    return classes, train_images, train_labels, val_images, val_labels

classes, train_images, train_labels, val_images, val_labels = setup('./data')

Generating images and labels ...
Number of images: 27000
Generating train / val splits ...
Training samples:  21600
Validation samples:  5400


In [4]:
import torch
from skimage import io 

class Dataset(torch.utils.data.Dataset):
    def __init__(self, images, labels):
        self.images = images
        self.labels = labels

    def __len__(self):
        return len(self.images)

    def __getitem__(self, ix):
        img = io.imread(self.images[ix])[...,(3,2,1)]
        img = torch.tensor(img / 4000, dtype=torch.float).clip(0,1).permute(2,0,1)  
        label = torch.tensor(self.labels[ix], dtype=torch.long)        
        return img, label
    
ds = {
    'train': Dataset(train_images, train_labels),
    'val': Dataset(val_images, val_labels)
}

batch_size = 1024
dl = {
    'train': torch.utils.data.DataLoader(ds['train'], batch_size=batch_size, shuffle=True, num_workers=20, pin_memory=True),
    'val': torch.utils.data.DataLoader(ds['val'], batch_size=batch_size, shuffle=False, num_workers=20, pin_memory=True)
}

In [5]:
import torch.nn.functional as F
import torchvision

class Model(torch.nn.Module):

    def __init__(self, n_outputs=10, use_amp=True):
        super().__init__()
        self.model = torchvision.models.resnet50(pretrained=True, progress=None)
        self.model.fc = torch.nn.Linear(2048, n_outputs)
        self.use_amp = use_amp

    def forward(self, x, log=False):
        if log:
            print(x.shape)
        with torch.cuda.amp.autocast(enabled=self.use_amp):
            return self.model(x)

In [6]:
from tqdm import tqdm
import numpy as np

def step(model, batch, device):
    x, y = batch
    x, y = x.to(device), y.to(device)
    y_hat = model(x)
    loss = F.cross_entropy(y_hat, y)
    acc = (torch.argmax(y_hat, axis=1) == y).sum().item() / y.size(0)
    return loss, acc

def train_amp(model, dl, optimizer, epochs=10, device="cpu", use_amp = True, prof=None, end=0):
    model.to(device)
    hist = {'loss': [], 'acc': [], 'val_loss': [], 'val_acc': []}
    scaler = torch.cuda.amp.GradScaler(enabled=use_amp)
    for e in range(1, epochs+1):
        # train
        model.train()
        l, a = [], []
        bar = tqdm(dl['train'])
        stop=False
        for batch_idx, batch in enumerate(bar):
            optimizer.zero_grad()
            
            # AMP
            with torch.cuda.amp.autocast(enabled=use_amp):
                loss, acc = step(model, batch, device)
            scaler.scale(loss).backward()
            # gradient clipping 
            #torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=0.1)
            scaler.step(optimizer)
            scaler.update()
            
            l.append(loss.item())
            a.append(acc)
            bar.set_description(f"training... loss {np.mean(l):.4f} acc {np.mean(a):.4f}")
            # profiling
            if prof:
                if batch_idx >= end:
                    stop = True
                    break
                prof.step()  
        hist['loss'].append(np.mean(l))
        hist['acc'].append(np.mean(a))
        if stop:
            break
        # eval
        model.eval()
        l, a = [], []
        bar = tqdm(dl['val'])
        with torch.no_grad():
            for batch in bar:
                loss, acc = step(model, batch, device)
                l.append(loss.item())
                a.append(acc)
                bar.set_description(f"evluating... loss {np.mean(l):.4f} acc {np.mean(a):.4f}")
        hist['val_loss'].append(np.mean(l))
        hist['val_acc'].append(np.mean(a))
        # log
        log = f'Epoch {e}/{epochs}'
        for k, v in hist.items():
            log += f' {k} {v[-1]:.4f}'
        print(log)
        
    return hist

In [7]:
model = Model()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
hist = train_amp(model, dl, optimizer, epochs=3, device="cuda")

Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to /root/.cache/torch/hub/checkpoints/resnet50-0676ba61.pth
  0%|          | 0/22 [00:00<?, ?it/s]/opt/conda/lib/python3.8/site-packages/torch/nn/functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  ../c10/core/TensorImpl.h:1153.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)
  0%|          | 0/22 [00:00<?, ?it/s]

Epoch 1/3 loss 0.3993 acc 0.8690 val_loss 3.7135 val_acc 0.6094


  0%|          | 0/22 [00:00<?, ?it/s]

Epoch 2/3 loss 0.1196 acc 0.9616 val_loss 0.3278 val_acc 0.8983


evluating... loss 0.1928 acc 0.9418: 100%|██████████| 6/6 [00:01<00:00,  3.08it/s]

Epoch 3/3 loss 0.0657 acc 0.9797 val_loss 0.1928 val_acc 0.9418


In [8]:
model = Model()
if torch.cuda.device_count() > 1:
  print("Let's use", torch.cuda.device_count(), "GPUs!")
  model = torch.nn.DataParallel(model)

Let's use 2 GPUs!


In [9]:
model.cuda()

# cada gpu recibe la mitad del batch !
output = model(torch.randn(32, 3, 32, 32).cuda(), log=True)

output.size()

torch.Size([16, 3, 32, 32])torch.Size([16, 3, 32, 32])



torch.Size([32, 10])

In [11]:
batch_size = 2048
dl = {
    'train': torch.utils.data.DataLoader(ds['train'], batch_size=batch_size, shuffle=True, num_workers=20, pin_memory=True),
    'val': torch.utils.data.DataLoader(ds['val'], batch_size=batch_size, shuffle=False, num_workers=20, pin_memory=True)
}

optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
hist = train_amp(model, dl, optimizer, epochs=3, device="cuda")

  0%|          | 0/11 [00:00<?, ?it/s]

Epoch 1/3 loss 0.5654 acc 0.8821 val_loss 7.8633 val_acc 0.3981


  0%|          | 0/11 [00:00<?, ?it/s]

Epoch 2/3 loss 0.1449 acc 0.9560 val_loss 0.8714 val_acc 0.8462


evluating... loss 0.2223 acc 0.9453: 100%|██████████| 3/3 [00:02<00:00,  1.07it/s]

Epoch 3/3 loss 0.0850 acc 0.9740 val_loss 0.2223 val_acc 0.9453


docker-compose up

docker-compose up -d

docker-compose down